In [ ]:
import os
import json
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# Путь к папке с изображениями
image_folder = 'D:/pictures/'

# Путь к JSON-файлу
json_file = 'D:/json1.json'

# Путь к папке для сохранения масок сегментации
output_folder = 'D:/masks/'
os.makedirs(output_folder, exist_ok=True)

# Загрузка аннотаций из JSON-файла с указанием правильной кодировки
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Получение списка файлов изображений
image_files = os.listdir(image_folder)

# Обход каждого изображения
for image_file in image_files:
    # Формирование пути к изображению
    image_path = os.path.join(image_folder, image_file)

    # Загрузка изображения
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Поиск соответствующих аннотаций для текущего изображения
    annotations = [annotation for annotation in data if annotation['filename'] == image_file]

    # Если для изображения нет аннотаций или regions пустой, пропускаем его
    if len(annotations[0]['regions']) == 0:
        continue

    # Создание пустой маски сегментации
    mask = np.zeros((image_height, image_width), dtype=np.uint8)

    # Обработка аннотаций
    for annotation in annotations:
        regions = annotation['regions']
        for region in regions:
            points_x = region['points_x']
            points_y = region['points_y']
            polygon_coords = np.vstack((points_x, points_y)).astype(np.int32).T
            cv2.fillPoly(mask, [polygon_coords], 255)

    # Сохранение маски сегментации
    mask_filename = os.path.splitext(image_file)[0]
    mask_path = os.path.join(output_folder, mask_filename)
    mask_image = Image.fromarray(mask)
    mask_image.save(mask_path)

    # Отображение изображения и маски
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image)
    axs[0].set_title('Изображение')
    axs[0].axis('off')
    axs[1].imshow(mask, cmap='gray')
    axs[1].set_title('Маска сегментации')
    axs[1].axis('off')
    plt.show()

    print(f"Создана маска сегментации для {image_file}")

print("Обработка завершена.")
